
# EDA on the 2014 Survey Dataset

## 01 Basic Overview of the Dataset

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import pie
import seaborn as sns
import plotly.express as px

In [ ]:
# Loading the dataset and overview
df = pd.read_csv("datasets/2014_survey.csv")
df.head()

In [ ]:
# Attribute names
df.columns

**Understanding the attributes**

The dataset contains the attributes which correspond to the following survey questions:
- ```Timestamp```
- ```Age```
- ```Gender```
- ```Country```
- ```state```: If you live in the United States, which state or territory do you live in?
- ```self_employed```: Are you self-employed?
- ```family_history```: Do you have a family history of mental illness?
- ```treatment```: Have you sought treatment for a mental health condition?
- ```work_interfere```: If you have a mental health condition, do you feel that it interferes with your work?
- ```no_employees```: How many employees does your company or organization have?
- ```remote_work```: Do you work remotely (outside of an office) at least 50% of the time?
- ```tech_company```: Is your employer primarily a tech company/organization?
- ```benefits```: Does your employer provide mental health benefits?
- ```care_options```: Do you know the options for mental health care your employer provides?
- ```wellness_program```: Has your employer ever discussed mental health as part of an employee wellness program?
- ```seek_help```: Does your employer provide resources to learn more about mental health issues and how to seek help?
- ```anonymity```: Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources?
- ```leave```: How easy is it for you to take medical leave for a mental health condition?
- ```mentalhealthconsequence```: Do you think that discussing a mental health issue with your employer would have negative consequences?
- ```physhealthconsequence```: Do you think that discussing a physical health issue with your employer would have negative consequences?
- ```coworkers```: Would you be willing to discuss a mental health issue with your coworkers?
- ```supervisor```: Would you be willing to discuss a mental health issue with your direct supervisor(s)?
- ```mentalhealthinterview```: Would you bring up a mental health issue with a potential employer in an interview?
- ```physhealthinterview```: Would you bring up a physical health issue with a potential employer in an interview?
- ```mentalvsphysical```: Do you feel that your employer takes mental health as seriously as physical health?
- ```obs_consequence```: Have you heard of or observed negative consequences for coworkers with mental health conditions in your workplace?
- ```comments```: Any additional notes or comments

In [ ]:
# Basic properties of the dataset
print("Shape:", df.shape)
df.describe()

In [ ]:
# View the properties of the attributes 
df.info()

In [ ]:
# Print all the columns with null values
print("Columns with null values:")
for col, i in zip(df.columns, df.isnull().sum()):
    if i != 0:
        print(col, ':', i)

## 02 Overview Analysis
From a high vantage point of the dataset, we determine that it consists of 1259 records and 27 attributes. The `Age` attribute is the only integer typed one, with all others being objects. `comments` has the most number of `null` values, far outnumbering the other attributes with `null` values - `state`, `work_interference` and `self_employed`. 

**Inferences and Thoughts**
- It makes sense that few participants would contribute to additional commenting. 
- The lack of answers in `state` could be attributed to people outside the US. Further calculation regarding the number of people who did live in the US and yet did not provide an answer here would yield the true `null` values.
- How many participants were undecided on the question of whether their mental health condition interferes with their work?

## 03 Data Preprocessing

### 1. Dealing with the `Age` attribute

In [ ]:
# Plot a histogram for the values of ages before preprocessing
plt.rcParams["figure.figsize"] = (10, 5)
age = sns.countplot(data=df, x='Age')
age.set_title('Analyzing the attribute Age before preprocessing')
age.set_xticklabels(age.get_xticklabels(), rotation=40, ha="right")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Get the unique values of ages
df['Age'].unique()

As we can analyze from the graph and unique values, there exist negative values of ages. Values below 18 and above 80 make very little sense, so we'll replace the corresponding values with the median of the age column.

In [ ]:
# How many dubious values of ages of each type do we have?
print("Responses with ages less than 18:")
print(df[df['Age'] < 18].Age)
print("\nResponses with ages more than 80:")
print(df[df['Age'] > 80].Age)
print("\nNull responses:", df['Age'].isnull().sum())

In [ ]:
# Replace values of ages less than 18 and more than 80 with the median

# Calculate previous median
median_without_dubious = df.loc[ (df['Age'] > 18) | (df['Age'] < 80), 'Age'].median()

# Replace dubious values with median
df.loc[ (df['Age'] < 18) | (df['Age'] > 80), 'Age'] = median_without_dubious

# Describe the attribute to analyze min and max
df['Age'].describe()

The minimum and maximum ages are 18 and 71 with the mean at around 32. We seemed to have completed cleaning up Age.

In [ ]:
# Plot a histogram for the values of ages after preprocessing
plt.rcParams["figure.figsize"] = (10, 5)
age = sns.countplot(data=df, x='Age')
age.set_title('Analyzing the attribute Age after preprocessing')
age.set_xticklabels(age.get_xticklabels(), rotation=40, ha="right")
plt.xticks(rotation=45)
plt.show()

**Analysis**

The graph for the continuous Age attribute is a **positively skewed histogram**. Many people fall into the age 30 and 33 bins.

In [ ]:
# Get the unique values of ages
print("Unique values:\n", df['Age'].unique())

### 2. Dealing with the `Gender` attribute

In [ ]:
# Are there any null values?
df['Gender'].isnull().sum()

Since there are no null values, we can move on.

In [ ]:
# What are the unique values in the Gender column?
df['Gender'].unique()

In the survey, this question was open-ended, and so the responses are strings that we need to classify into bins. We do so with three bins:
- Male
- Female
- Other

`A little about you` and `p` certainly do not belong here. We remove the records with those attribute values.

In [ ]:
# Remove the records with those attribute values
df = df[~df['Gender'].isin(['A little about you', 'p'])]

In [ ]:
# Replace the strings with the values of bins we require
# Group all related strings into bins

# Group into Male bin
df['Gender'].replace(['Male ', 'male', 'M', 'm', 'Male', 'Cis Male','Man', 'cis male', 'Mail', 'Male-ish', 
                      'Male (CIS)', 'Cis Man', 'msle', 'Malr', 'Mal', 'maile', 'Make']
                     
                     , 'Male', inplace = True)

# Group into Female bin
df['Gender'].replace(['Female ', 'female', 'F', 'f', 'Woman', 'Female', 'femail', 'Cis Female', 
                      'cis-female/femme', 'Femake', 'Female (cis)', 'woman'], 
                     
                     'Female', inplace = True)

# Group into Other bin
df["Gender"].replace(['Female (trans)', 'queer/she/they', 'non-binary','fluid', 'queer','Androgyne',
                      'Trans-female', 'male leaning androgynous', 'Agender', 'Nah', 'All', 
                      'ostensibly male, unsure what that really means', 'Genderqueer', 'Enby', 
                      'Neuter', 'something kinda male?', 'Guy (-ish) ^_^', 'Trans woman'],
                     
                     'Other', inplace = True)

df['Gender'].describe()

In [ ]:
# Viewing the counts of each type
df['Gender'].value_counts()

In [ ]:
# Visualizing the proportion of genders
fig, ax = plt.subplots()
labels = ['Male', 
         'Female',
         'Other']
male_perc = (df['Gender'] == 'Male').sum() / df['Gender'].count()
female_perc = (df['Gender'] == 'Female').sum() / df['Gender'].count()
other_perc = (df['Gender'] == 'Other').sum() / df['Gender'].count()

percentages = [male_perc, female_perc, other_perc]

ax.pie(percentages, labels=labels,  
       autopct='%1.0f%%', 
       shadow=False, startangle=0,   
       pctdistance=1.2,labeldistance=1.4)
ax.axis('equal')
ax.set_title("Gender values after Preprocessing")
ax.legend(frameon=False, bbox_to_anchor=(1.5,0.8))

As the plot suggests, male responders far outnumber the female and other responders. This is a point to keep in mind as we work on our models in the future.

### 4. Dealing with the `comments` attribute

Viewing comments after droping rows with missing values

In [ ]:
for count, i in enumerate(df['comments'].dropna()):
    if count < 30:
        print(i[:80])

## 04 Data Visualization

### 1. Analysis of the `leave` attribute

In [ ]:
df['leave'].value_counts()

In the below plot column 'leave' is grouped and is represented as a rectangular mark.

In [ ]:
fig, ax = plt.subplots(figsize = (12,6))
ind = np.arange(len(df['leave']))
sns.barplot(x=df['leave'].value_counts().index, 
            y = df['leave'].value_counts(),  
            data = df, 
            ax=ax)
ax.set_title("Ease of taking leaves")
ax.set_ylabel("Number of respondents")
ax.set_ylim(0,100)
ax.set_xlabel("Ease of taking a leave for a mental health condition")
ax.set_xticks(range(0,len(ind)))
ax.set_xticklabels(list(df['leave']), rotation=45)

People with family history of metal illnes  grouped by treatment sought

In [ ]:
temp_grp_df = df.groupby(['family_history','treatment']).count()
temp_grp_df.reset_index(drop=False, inplace=True)

fig = px.bar(temp_grp_df, x='family_history', y='Age', labels={'Age':'Count'}, orientation='v', color='treatment',title='People with Family History of Mental Illness Grouped by Treatment Sought' ,barmode='group')
fig.show()

In [ ]:
temp_grp_df = df.groupby(['mental_health_consequence','phys_health_consequence','mental_vs_physical']).count()
temp_grp_df.reset_index(drop=False, inplace=True)

fig = px.bar(temp_grp_df, x='mental_health_consequence', y='Age', labels={'Age':'Count'}, orientation='v', color='phys_health_consequence', facet_col='mental_vs_physical',barmode='group')
fig.show()

In [ ]:
df.groupby('Gender').Age.plot(kind='kde')

In [ ]:
df.groupby('Gender').Age.hist()

- When the argument of color corresponds to non-numerical data(here yes/no), discrete colors are used.
- Column - country is grouped and is represented as a sector of the treemap.

In [ ]:
temp_grp_df = df.groupby(['Country','treatment']).count()
temp_grp_df.reset_index(drop=False, inplace=True)

fig = px.treemap(temp_grp_df,path=['Country', 'treatment'],  color='treatment', values='Age',labels={'Age':'Count'})
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [ ]:
def bar_plot(categ_var):
    """
        input: variable ex: "anonymity"
        output: bar plot & value count
    """
    # get feature
    var = df[categ_var]
    # count number of categorical variable
    var_value = var.value_counts()
    
    plt.figure(figsize = (9,3))
    plt.bar(var_value.index, var_value)
    plt.xticks(var_value.index, var_value.values)
    plt.ylabel("Frequency")
    plt.title(categ_var)
    plt.show()
    print(f"{categ_var}:\n{var_value}")

Bar plot for self employed,family_history.tech_company,phys_health_consquence,supervisor

In [ ]:
categorical_vars = ["self_employed","family_history","tech_company","phys_health_consequence","supervisor"]
for c in categorical_vars:
    bar_plot(c)

Treatment percentage for a mental health condition

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize = (7,5))
eda_percentage = df['treatment'].value_counts(normalize = True).rename_axis('treatment').reset_index(name = 'Percentage')
sns.barplot(x = 'treatment', y = 'Percentage', data = eda_percentage.head(10))
plt.title('Get Treatment of Survey Respondents')
plt.show()

Difference in number of respondents that get treatment and no treatment(based of gender).

In [ ]:
plt.figure(figsize = (20,5))
plt.title('Gender of Survey Respondents')
plt.subplot(1,2,2)
sns.countplot(x=df['Gender'], hue = df['treatment'])
plt.title('Gender of Survey Respondents')
plt.show()

##COUNTRY COLUMN

In [ ]:
country_count = df.Country.value_counts().sort_values(ascending=False).to_frame()[:10]
country_count = country_count.rename(columns={'Country': 'count'})
plt.figure(figsize=(15,5))
ax = sns.barplot(x=country_count.index, y='count', data=country_count,  palette="ch:.25")
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
ax = ax.set_title('Top 10 countries')

## STATE COLUMN

In [ ]:
state_count = df.state.value_counts().sort_values(ascending=False).to_frame()[:10]
state_count = state_count.rename(columns={'state': 'count'})
plt.figure(figsize=(10,10))
ax = sns.barplot(x=state_count.index, y='count', data=state_count,  palette="ch:.25")
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
ax = ax.set_title('Top 10 States')

##NO OF EMPLOYEES COLUMN

In [ ]:
df['no_employees'].value_counts()


##TREATEMENT COLUMN

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize = (8,5))
plt.title('Get Treatment of Survey Respondents', fontsize=18, fontweight='bold')
eda_percentage = df['treatment'].value_counts(normalize = True).rename_axis('treatment').reset_index(name = 'Percentage')

ax = sns.barplot(x = 'treatment', y = 'Percentage', data = eda_percentage.head(10), palette='Dark2')
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy() 
    ax.annotate(f'{height:.0%}', (x + width/2, y + height*1.02), ha='center', fontweight='bold')


##WORK_INTERFERE COLUMN

In [ ]:
plt.figure(figsize = (20,6))
plt.subplot(1,2,1)
eda_percentage = df['work_interfere'].value_counts(normalize = True).rename_axis('work_interfere').reset_index(name = 'Percentage')
ax = sns.barplot(x = 'work_interfere', y = 'Percentage', data = eda_percentage, palette='Dark2')
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy() 
    ax.annotate(f'{height:.0%}', (x + width/2, y + height*1.02), ha='center', fontweight='bold')

plt.title('Work Interference of Survey Respondents', fontsize=18, fontweight='bold')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
#plt.legend(fontsize=16)

plt.subplot(1,2,2)
sns.countplot(x=df['work_interfere'], hue = df['treatment'],  palette=dict(No = '#d95f02', Yes = '#1b9e77'))
plt.title('Work Interference of Survey Respondents', fontsize=18, fontweight='bold')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=16)

##WELLNESS PROGRAM COLUMN

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
eda_percentage = df['wellness_program'].value_counts(normalize = True).rename_axis('wellness_program').reset_index(name = 'Percentage')
ax = sns.barplot(x = 'wellness_program', y = 'Percentage', data = eda_percentage, palette='Dark2')
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy() 
    ax.annotate(f'{height:.0%}', (x + width/2, y + height*1.02), ha='center', fontweight='bold')

plt.title('Mental Health in Wellness Program provided to the Employees', fontsize=16, fontweight='bold')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
#plt.legend(fontsize=16)

plt.subplot(1,2,2)
sns.countplot(x=df['wellness_program'], hue = df['treatment'], palette='Dark2')
plt.title('Mental Health in Wellness Program provided to the Employees', fontsize=16, fontweight='bold')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=16)